# `Практикум по программированию на языке Python`
<br>

## `Занятие 12: Flask`
<br><br>

### `Максим Находнов (nakhodnov17@gmail.com)`

#### `Москва, 2023`

О чём можно узнать из этого ноутбука:

* Пример серверной библиотеки для создания WEB-приложений: Flask
* REST API

Источники материалов:

* [Flask](https://github.com/mmp-practicum-team/mmp_practicum_fall_2022/tree/main/Seminars/12-flask)
* [Flask WSGI](https://github.com/mmp-practicum-team/mmp_practicum_fall_2022/blob/main/Seminars/13-web-servers/13-web-servers.ipynb)
* [REST](https://habr.com/ru/post/483202/), [REST Flask](https://habr.com/ru/post/246699/)

#### `Flask`

Flask — микрофреймворк для создания вебсайтов на языке Python.

##### `Простейший пример сервера`

In [1]:
%%writefile simple_flask.py
# Простейший сервер на flask
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'
   
if __name__ == '__main__':
    app.run(host='localhost')

Writing simple_flask.py


In [14]:
import requests

In [15]:
r = requests.get('http://127.0.0.1:5000/')
print(r.status_code)
print(r.headers)
print(r.content)

200
{'Server': 'Werkzeug/2.2.2 Python/3.10.6', 'Date': 'Thu, 26 Jan 2023 14:48:32 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '13', 'Connection': 'close'}
b'Hello, World!'


##### `Простейший пример сервера. POST/GET`

По умолчанию `route` отвечает только на `GET` запросы.

Если нужно, можно явно добавить HTTP-методы, которые будут обрабатываться

In [16]:
%%writefile simple_flask_post.py
# Простейший сервер на flask
from flask import Flask

app = Flask(__name__)

@app.route('/post', methods=['POST'])
def hello_path():
    return 'Hello, Path!'
   
if __name__ == '__main__':
    app.run(host='localhost')

Writing simple_flask_post.py


In [17]:
r = requests.post('http://127.0.0.1:5000/post')
print(r.status_code)
print(r.headers)
print(r.content)

200
{'Server': 'Werkzeug/2.2.2 Python/3.10.6', 'Date': 'Thu, 26 Jan 2023 14:48:46 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '12', 'Connection': 'close'}
b'Hello, Path!'


In [18]:
r = requests.get('http://127.0.0.1:5000/post')
print(r.status_code)
print(r.headers)
print(r.content)

405
{'Server': 'Werkzeug/2.2.2 Python/3.10.6', 'Date': 'Thu, 26 Jan 2023 14:48:47 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Allow': 'POST, OPTIONS', 'Content-Length': '153', 'Connection': 'close'}
b'<!doctype html>\n<html lang=en>\n<title>405 Method Not Allowed</title>\n<h1>Method Not Allowed</h1>\n<p>The method is not allowed for the requested URL.</p>\n'


##### `Простейший пример сервера. Variable Rules`

В пути можно использовать переменные

Синтаксис: `<converter:variable_name>`

Доступные converters:
- `string`
- `int`
- `float`
- `path`
- `uuid`

In [19]:
%%writefile simple_flask_converters.py
from flask import Flask

app = Flask(__name__)

@app.route('/hello/<string:name>')
def hello_name(name):
    return f'Hello {name}!'
   
if __name__ == '__main__':
    app.run(host='localhost')

Writing simple_flask_converters.py


In [20]:
r = requests.get('http://127.0.0.1:5000/hello/John')
print(r.content)

b'Hello John!'


#### `Flask WSGI`

Flask используется для разработки и отладки.

Для промышленной эксплуатации необходимо использование **WSGI** (Web Server Gateway Interface) сервера:
- WSGI-сервера были разработаны чтобы обрабатывать множество запросов одновременно. А фреймворки (в том числе Flask) не предназначены для обработки тысяч запросов и не дают решения того, как наилучшим образом маршрутизировать запросы с веб-сервера
- с WSGI не нужно беспокоиться о том, как ваша конкретная инфраструктура использует стандарт WSGI
- WSGI дает Вам гибкость в изменении компонентов веб-стека без изменения приложения, которое работает с WSGI

Здесь WSGI (Web Server Gateway Interface) — стандарт взаимодействия между Python-программой, выполняющейся на стороне сервера, и самим веб-сервером, например Apache. Фактически, это интерпретатор Python, который запускает WSGI-приложение, написанное на Flask.

При поступлении запроса активизируется WSGI-приложение, выполняется определенный обработчик, который еще называется «Представление» и реализованный в виде функции на языке Python. Соответственно, если приходит сразу несколько запросов, то одна и та же функция-обработчик может быть запущена в параллельных потоках. Многопоточность – это норма для фреймворков, поэтому, работая с представлениями во Flask, всегда следует это учитывать.

Если не планируется большой нагрузки, для `flask` это может быть `waitress`.

Установка: `pip install waitress`

Использование:

In [21]:
%%writefile flask_wsgi.py
import time
from flask import Flask
from waitress import serve

app = Flask(__name__)

@app.route('/')
def hello_world():
    time.sleep(5)
    return 'Hello, World!'
   
if __name__ == '__main__':
    # Вместо запуска flask запускаем waitress.serve
#     app.run(host='localhost', threaded=False)
    serve(app, host='localhost', port='5000', threads=2)

Writing flask_wsgi.py


Либо запускаем из командной строки: 
```bash
waitress-serve --port 5000 '<имя модуля>:<перемнная приложения>'
```

Если наш файл называется `server.py`, то наш пример можно запустить командой: 
```bash
waitress-serve --port 5000 'server:app'
```

### REST API

#### `Что такое REST?`

**REST** (Representational State Transfer — «передача репрезентативного состояния» или «передача „самоописываемого“ состояния») — архитектурный стиль взаимодействия компонентов распределённого приложения в сети. Другими словами, REST — это **набор правил** того, как программисту организовать написание **кода серверного приложения**, чтобы все системы легко **обменивались данными** и приложение можно было масштабировать

#### `Правила REST`

1. **Клиент-Сервер**: Должно быть разделение между сервером, который предлагает сервис и клиентом, который использует ее.
2. **Stateless**: Каждый запрос от клиента должен содержать всю информацию, необходимую серверу для выполнения запроса. Другими словами, сервер не обязан сохранять информацию о состоянии клиента.
3. **Кэширование**: В каждом запросе клиента должно явно содержаться указание о возможности кэширования ответа и получения ответа из существующего кэша.
4. **Уровневая система**: Клиент может взаимодействовать не напрямую с сервером, а с произвольным количеством промежуточных узлов. При этом клиент может не знать о существовании промежуточных узлов, за исключением случаев передачи конфиденциальной информации.
5. **Унификация**: Унифицированный программный интерфейс сервера.
6. **Код по запросу**: Сервера могут поставлять исполняемый код или скрипты для выполнения их на стороне клиентов.

#### `Ресурс`

**Ресурс** — это ключевая абстракция, на которой концентрируется протокол HTTP. Ресурс — это все, что вы хотите показать внешнему миру через ваше приложение. Например, если мы пишем приложение для управления задачами, экземпляры ресурсов будут следующие:
* Конкретный пользователь
* Конкретная задача
* Список задач

#### `Дизайн REST`

| Метод HTTP |            Действие           |                                      Пример                                     |
|:----------:|:-----------------------------:|:-------------------------------------------------------------------------------:|
| **GET**        | Получить информацию о ресурсе | `example.com/api/orders` (получить список заказов)                                |
| **GET**        | Получить информацию о ресурсе | `example.com/api/orders/123` (получить заказ #123)                                |
| **POST**       | Создать новый ресурс          | `example.com/api/orders` (создать новый заказ из данных переданных с запросом)    |
| **PUT**        | Обновить ресурс               | `example.com/api/orders/123` (обновить заказ #123 данными переданными с запросом) |
| **DELETE**     | Удалить ресурс                | `example.com/api/orders/123` (удалить заказ #123)                                 |

Дизайн REST не дает рекомендаций каким конкретно должен быть формат данных передаваемых с запросами. Данные переданные в теле запроса могут быть JSON blob, или с помощью аргументов в URL.

#### `REST. Пример на Flaks`

| Метод HTTP |                       URI                       |           Действие           |
|:----------:|:-----------------------------------------------:|:----------------------------:|
| **GET**        | `http://[hostname]/todo/api/v1.0/tasks`           | Получить список задач        |
| **GET**        | `http://[hostname]/todo/api/v1.0/tasks/[task_id]` | Получить задачу              |
| **POST**       | `http://[hostname]/todo/api/v1.0/tasks`           | Создать новую задачу         |
| **PUT**        | `http://[hostname]/todo/api/v1.0/tasks/[task_id]` | Обновить существующую задачу |
| **DELETE**     | `http://[hostname]/todo/api/v1.0/tasks/[task_id]` | Удалить задачу               |

Наша задача будет иметь следующие поля:

* **id** — уникальный идентификатор задачи. Тип int.
* **title** — Краткое описание задачи. Тип str.
* **description** — подробное описание задачи. Тип str.
* **done** — отметка о выполнении. Тип bool.

In [22]:
%%writefile flask_rest_api.py

from flask import Flask, jsonify, abort, request, make_response, url_for

app = Flask(__name__)

tasks = [
    {
        'id': 1,
        'title': u'Buy groceries',
        'description': u'Milk, Cheese, Pizza, Fruit, Tylenol', 
        'done': False
    },
    {
        'id': 2,
        'title': u'Learn Python',
        'description': u'Need to find a good Python tutorial on the web', 
        'done': False
    }
]

@app.route('/todo/api/v1.0/tasks', methods=['GET'])
def get_tasks():
    return {'tasks': tasks}


@app.route('/todo/api/v1.0/tasks/<int:task_id>', methods=['GET'])
def get_task(task_id):
    for task in tasks:
        if task['id'] == task_id:
            return {'task': task}
    abort(404)


@app.route('/todo/api/v1.0/tasks', methods=['POST'])
def create_task():
    if not request.json or not 'title' in request.json:
        abort(400)
    task = {
        'id': tasks[-1]['id'] + 1,
        'title': request.json['title'],
        'description': request.json.get('description', ""),
        'done': False
    }
    tasks.append(task)
    return jsonify({'task': task}), 201


@app.route('/todo/api/v1.0/tasks/<int:task_id>', methods=['PUT'])
def update_task(task_id):
    target_tasks = [task for task in tasks if task['id'] == task_id]
    if len(target_tasks) != 1:
        abort(404)
    task = target_tasks[0]
        
    if not request.json:
        abort(400)
    if 'title' in request.json and not isinstance(request.json['title'], str):
        abort(400)
    if 'description' in request.json and not isinstance(request.json['description'], str):
        abort(400)
    if 'done' in request.json and not isinstance(request.json['done'], bool):
        abort(400)

    task['title'] = request.json.get('title', task['title'])
    task['description'] = request.json.get('description', task['description'])
    task['done'] = request.json.get('done', task['done'])
    
    return jsonify({'task': task})
    
    
@app.route('/todo/api/v1.0/tasks/<int:task_id>', methods = ['DELETE'])
def delete_task(task_id):
    target_tasks = [task for task in tasks if task['id'] == task_id]
    if len(target_tasks) != 1:
        abort(404)
    task = target_tasks[0]
    
    tasks.remove(task)
    return {'result': True}


if __name__ == '__main__':
    app.run(debug=True)

Writing flask_rest_api.py


##### `GET tasks`

In [23]:
r = requests.get('http://127.0.0.1:5000/todo/api/v1.0/tasks')
print(r.status_code)

content = json.loads(r.content)
print(json.dumps(content, indent=4, sort_keys=True))

200
{
    "tasks": [
        {
            "description": "Milk, Cheese, Pizza, Fruit, Tylenol",
            "done": false,
            "id": 1,
            "title": "Buy groceries"
        },
        {
            "description": "Need to find a good Python tutorial on the web",
            "done": false,
            "id": 2,
            "title": "Learn Python"
        }
    ]
}


##### `GET specific task`

In [24]:
r = requests.get('http://127.0.0.1:5000/todo/api/v1.0/tasks/1')
# r = requests.get(content['tasks'][0]['url'])

print(r.status_code)

content = json.loads(r.content)
print(json.dumps(content, indent=4, sort_keys=True))

200
{
    "task": {
        "description": "Milk, Cheese, Pizza, Fruit, Tylenol",
        "done": false,
        "id": 1,
        "title": "Buy groceries"
    }
}


##### `POST new task`

In [25]:
r = requests.post('http://127.0.0.1:5000/todo/api/v1.0/tasks', json={'title': 'New Task', 'description': 'New info'})

print(r.status_code)

content = json.loads(r.content)
print(json.dumps(content, indent=4, sort_keys=True))

201
{
    "task": {
        "description": "New info",
        "done": false,
        "id": 3,
        "title": "New Task"
    }
}


In [26]:
r = requests.get('http://127.0.0.1:5000/todo/api/v1.0/tasks')
print(r.status_code)

content = json.loads(r.content)
print(json.dumps(content, indent=4, sort_keys=True))

200
{
    "tasks": [
        {
            "description": "Milk, Cheese, Pizza, Fruit, Tylenol",
            "done": false,
            "id": 1,
            "title": "Buy groceries"
        },
        {
            "description": "Need to find a good Python tutorial on the web",
            "done": false,
            "id": 2,
            "title": "Learn Python"
        },
        {
            "description": "New info",
            "done": false,
            "id": 3,
            "title": "New Task"
        }
    ]
}


##### `PUT changes to the tasks`

In [27]:
r = requests.put('http://127.0.0.1:5000/todo/api/v1.0/tasks/1', json={'done': True})
print(r.status_code)

content = json.loads(r.content)
print(json.dumps(content, indent=4, sort_keys=True))

200
{
    "task": {
        "description": "Milk, Cheese, Pizza, Fruit, Tylenol",
        "done": true,
        "id": 1,
        "title": "Buy groceries"
    }
}


In [28]:
r = requests.get('http://127.0.0.1:5000/todo/api/v1.0/tasks')
print(r.status_code)

content = json.loads(r.content)
print(json.dumps(content, indent=4, sort_keys=True))

200
{
    "tasks": [
        {
            "description": "Milk, Cheese, Pizza, Fruit, Tylenol",
            "done": true,
            "id": 1,
            "title": "Buy groceries"
        },
        {
            "description": "Need to find a good Python tutorial on the web",
            "done": false,
            "id": 2,
            "title": "Learn Python"
        },
        {
            "description": "New info",
            "done": false,
            "id": 3,
            "title": "New Task"
        }
    ]
}


##### `DELETE a task`

In [29]:
r = requests.delete('http://127.0.0.1:5000/todo/api/v1.0/tasks/1')
print(r.status_code)

content = json.loads(r.content)
print(json.dumps(content, indent=4, sort_keys=True))

200
{
    "result": true
}


In [30]:
r = requests.get('http://127.0.0.1:5000/todo/api/v1.0/tasks')
print(r.status_code)

content = json.loads(r.content)
print(json.dumps(content, indent=4, sort_keys=True))

200
{
    "tasks": [
        {
            "description": "Need to find a good Python tutorial on the web",
            "done": false,
            "id": 2,
            "title": "Learn Python"
        },
        {
            "description": "New info",
            "done": false,
            "id": 3,
            "title": "New Task"
        }
    ]
}


In [31]:
r = requests.delete('http://127.0.0.1:5000/todo/api/v1.0/tasks/0')
print(r.status_code)

404


#### `REST. Пример на Flaks. V2.0`

In [32]:
%%writefile flask_rest_api_v2.py

from flask import Flask, jsonify, abort, request, make_response, url_for

app = Flask(__name__)

@app.errorhandler(400)
def bad_request(error):
    return make_response(jsonify( { 'error': error.description } ), 400)

@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify( { 'error': error.description } ), 404)


tasks = [
    {
        'id': 1,
        'title': u'Buy groceries',
        'description': u'Milk, Cheese, Pizza, Fruit, Tylenol', 
        'done': False
    },
    {
        'id': 2,
        'title': u'Learn Python',
        'description': u'Need to find a good Python tutorial on the web', 
        'done': False
    }
]

def make_public_task(task):
    return (
        task | 
        {'url' : url_for('get_task', task_id=task['id'], _external=True)}
    )


@app.route('/todo/api/v2.0/tasks', methods=['GET'])
def get_tasks():
    return {
        'tasks': [make_public_task(task) for task in tasks]
    }


@app.route('/todo/api/v2.0/tasks/<int:task_id>', methods=['GET'])
def get_task(task_id):
    for task in tasks:
        if task['id'] == task_id:
            return {'task': make_public_task(task)}
    abort(404, 'Task not found')


@app.route('/todo/api/v2.0/tasks', methods=['POST'])
def create_task():
    if not request.json or not 'title' in request.json:
        abort(400, 'Not a json')
    task = {
        'id': tasks[-1]['id'] + 1,
        'title': request.json['title'],
        'description': request.json.get('description', ""),
        'done': False
    }
    tasks.append(task)
    return jsonify({'task': make_public_task(task)}), 201


@app.route('/todo/api/v2.0/tasks/<int:task_id>', methods=['PUT'])
def update_task(task_id):
    target_tasks = [task for task in tasks if task['id'] == task_id]
    if len(target_tasks) != 1:
        abort(404, 'Task not found')
    task = target_tasks[0]
        
    if not request.json:
        abort(400, 'Not a json')
    if 'title' in request.json and not isinstance(request.json['title'], str):
        abort(400, 'Bad type for title')
    if 'description' in request.json and not isinstance(request.json['description'], str):
        abort(400, 'Bad type for description')
    if 'done' in request.json and not isinstance(request.json['done'], bool):
        abort(400, 'Bad type for done')

    task['title'] = request.json.get('title', task['title'])
    task['description'] = request.json.get('description', task['description'])
    task['done'] = request.json.get('done', task['done'])
    return jsonify({'task': make_public_task(task)})
    
    
@app.route('/todo/api/v2.0/tasks/<int:task_id>', methods = ['DELETE'])
def delete_task(task_id):
    target_tasks = [task for task in tasks if task['id'] == task_id]
    if len(target_tasks) != 1:
        abort(404, 'Task not found')
    task = target_tasks[0]
    
    tasks.remove(task)
    return {'result': True}


if __name__ == '__main__':
    app.run(debug=True)

Writing flask_rest_api_v2.py


##### `Более удобный интерфейс идентификаторов`

In [33]:
r = requests.get('http://127.0.0.1:5000/todo/api/v2.0/tasks')
print(r.status_code)

content = json.loads(r.content)
print(json.dumps(content, indent=4, sort_keys=True))

200
{
    "tasks": [
        {
            "description": "Milk, Cheese, Pizza, Fruit, Tylenol",
            "done": false,
            "id": 1,
            "title": "Buy groceries",
            "url": "http://127.0.0.1:5000/todo/api/v2.0/tasks/1"
        },
        {
            "description": "Need to find a good Python tutorial on the web",
            "done": false,
            "id": 2,
            "title": "Learn Python",
            "url": "http://127.0.0.1:5000/todo/api/v2.0/tasks/2"
        }
    ]
}


In [34]:
target_task_url = content['tasks'][0]['url']
r = requests.get(target_task_url)

print(r.status_code)

content = json.loads(r.content)
print(json.dumps(content, indent=4, sort_keys=True))

200
{
    "task": {
        "description": "Milk, Cheese, Pizza, Fruit, Tylenol",
        "done": false,
        "id": 1,
        "title": "Buy groceries",
        "url": "http://127.0.0.1:5000/todo/api/v2.0/tasks/1"
    }
}


In [35]:
r = requests.delete(target_task_url)
print(r.status_code)

200


##### `Более понятные ошибки`

In [36]:
r = requests.put(target_task_url, json={'done': True})
print(r.status_code)

content = json.loads(r.content)
print(json.dumps(content, indent=4, sort_keys=True))

404
{
    "error": "Task not found"
}
